### 1.1 Min-Max Normalization 데이터 정규화 작업

In [ ]:
# 텐서플로우 2.0 불러오기
import tensorflow as tf
print(tf.__version__)

In [ ]:
# 데이터 불러오기 : fashion_mnist - 의류 이미지 / 10개의 범주 / 단일 채널 / 28 * 28
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()

In [ ]:
# 데이터의 수 확인하기
print(len(train_X), len(test_X))

In [ ]:
# 이미지의 크기 (28 * 28) 확인 가능
print(train_X.shape, test_X.shape)

In [ ]:
# 이미지 확인
# Label : 0.티셔츠,상의 / 1.바지 / 2.스웨터 / 3.드레스 / 4.코트 / 5.샌들 / 6.셔츠 / 7.운동화 / 8.가방 / 9.부츠 
import matplotlib.pyplot as plt
plt.imshow(train_X[2], cmap='pink') # cmap을 통해 이미지의 출력 색상 선택
plt.colorbar() # 우측에 색상값의 정보를 bar 형태로 출력 -> 색상값은 0 ~ 255 의 값을 가지고 있음을 확인
plt.show()

In [ ]:
# 정답 범주 확인
print('정답 범주 : ', train_Y[2]) # 확인 결과 2번 이미지는 0번(티셔츠/상의)

In [ ]:
# 데이터 정규화 이전의 '이미지 픽셀 행렬'
print(train_X[2]) # 아직은 큰 의미를 찾아볼 수 없다.

In [ ]:
# 위의 작업을 통해 이미지의 최솟값은 0, 최댓값은 255임을 확인.
# 최대-최소 정규화는 데이터를 정규화하는 가장 일반적인 방법 : (X - min) / (max - min)
train_X = train_X / 255.0
test_X = test_X / 255.0

print(train_X[2])

### 1.2 다층 퍼셉트론에서의 이미지 처리

In [ ]:
# Flatten()은 다차원 이미지를 1차원으로 평평하게 바꿔주는 단순 레이어 (input_shape : 원본 데이터의 크기를 입력)
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28, 28)), # 원본 데이터가 28*28 의 2차원 array
                             tf.keras.layers.Dense(units=128, activation='relu'), # Dense 레이어는 완전연결 레이어
                             tf.keras.layers.Dense(units=10, activation='softmax') # 마지막 레이어의 units 값은 정답 범주의 수와 동일
])

In [ ]:
# 컴파일 단계
model.compile(optimizer=tf.keras.optimizers.Adam(), # Adam의 기본 인자값 : lr=0.0001
              loss='sparse_categorical_crossentropy', # 희소행렬(대부분의 값이 0인 행렬)을 나타내는 데이터를 전처리 없이 정답행렬로 사용 가능
              metrics=['accuracy'])

model.summary() # 층 수가 적음에도 총 파라미터 수가 굉장히 많이 증가함을 확인할 수 있음

In [ ]:
# 신경망 학습 단계 / EarlyStopping으로 일정 부분 과적합 해소 시도
history = model.fit(train_X, train_Y, epochs=500, validation_split=0.25, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')])

In [ ]:
# 정확도 및 손실값에 대한 결과 시각화 : 검증 데이터와 학습 데이터 비교 / 과적합 혹은 과소적합 확인
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label = 'loss')
plt.plot(history.history['val_loss'], 'r--', label = 'val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label = 'accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label = 'val_accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.show() # 과적합이 있음을 포착할 수 있음

In [ ]:
# 최종 평가 : evaluate() - 최종 정확도 및 손실값 확인
model.evaluate(test_X, test_Y) # loss와 accuracy 모두 좋지 못한 수치가 나옴 -> 이를 기준으로 이후 진행할 실험들과 비교

In [ ]:
# 예측 : predict() - 학습된 신경망을 활용하여 예측을 수행
pred_X = model.predict(test_X[[1]]) # 2차원으로 넣는 것에 주의

#예측 이미지 확인
plt.imshow(test_X[1], cmap='pink')
plt.show()

# 예측 결과 확인
print(pred_X+1) # 임의의 값을 더하여 출력한다.
print('예측 수행 범주 : ', test_Y[1]) # 2번째 범주일 확률이 가장 높으므로 2가 출력

### 1.3 첫 번째 실험 - 합성곱 계층만 활용

In [ ]:
# 합성곱 연산을 진행하기 위한 4차원 (미니배치 데이터, 입력 이미지 행, 입력 이미지 열, 입력 채널 수)
# reshape()을 통해 기존 3차원에서 채널이 추가된 4차원 형태로 차원 변경
print(train_X.shape, test_X.shape)

train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)
print(train_X.shape, test_X.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))

for i in range(16):
    plt.subplot(4, 4, i+1) # (행, 열, 순서)
    plt.imshow(train_X[i].reshape(28, 28), cmap='pink') # reshape()를 통해 기존 4차원에서 다시 2차원으로 변경하여 이미지 출력 (시각화)

plt.show()
print(train_Y[:16])

In [ ]:
# 실험 1 - 풀링 레이어 없이, 단순 합성곱 계층만 활용
model= tf.keras.Sequential([
                            tf.keras.layers.Conv2D(input_shape=(28, 28, 1), # 입력 이미지의 크기 및 채널
                                                   kernel_size=(3,3), # 커널 사이즈
                                                   filters=16, # 몇 개의 채널(필터)을 생성할 것인지
                                                   strides=(1, 1), # 건너 뛰기 (기본값 : 1, 1)
                                                   padding = 'valid'), # 패딩 방식 (기본값 : 'valid' - 패딩 없이 축소한 상태 그대로)
                            tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32), # filters의 값은 이전보다 점차 크게 설정해야한다.
                            tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64),
                            tf.keras.layers.Flatten(), # 다차원 레이어를 1차원으로 펼쳐준다.

                            tf.keras.layers.Dense(units=128, activation='relu'), # 실험을 위해 기존의 것과 동일하게 설정
                            tf.keras.layers.Dense(units=10, activation='softmax') # 실험을 위해 기존의 것과 동일하게 설정
])

In [ ]:
# 컴파일 단계
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary() # '풀링 계층'이 없기 때문에 약 400만개의 매우 많은 파라미터 생성 (너무 많음) 

In [ ]:
# 신경망 학습 단계
history = model.fit(train_X, train_Y, epochs=500, validation_split=0.25, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')])

In [ ]:
# 정확도 및 손실값에 대한 결과 시각화
plt.figure(figsize = (12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2) 
plt.plot(history.history['accuracy'], 'g-', label='acc')
plt.plot(history.history['val_accuracy'], 'k--',label='val_acc')
plt.xlabel('Epoch')
plt.legend()

plt.show() # 과적합에 대한 문제가 지속됨을 확인

In [ ]:
# 최종 평가
# 하이퍼 파라미터는 동일 / 정확도 및 손실값이 어느 정도 향상 / 과적합 현상 발생
# 퍼셉트론 수가 너무 많다보니 학습에 많은 시간 소요 (기회비용 손해)
model.evaluate(test_X, test_Y)

### 두 번째 실험 - 합성곱, 풀링, 드랍아웃 활용

In [ ]:
# 합성곱 계층과 풀링 계층이 번갈아 가며 등장
model= tf.keras.Sequential([
                            tf.keras.layers.Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=32),
                            tf.keras.layers.MaxPool2D(strides=(2, 2), pool_size=(2, 2)), # 최대치 풀링 : 건너뛰기 (보폭) / 풀링 사이즈 설정 (기본값 : 2, 2)
                            tf.keras.layers.Conv2D(kernel_size=(3, 3),filters=64),
                            tf.keras.layers.AvgPool2D(strides=(2, 2)), # 평균치 풀링
                            tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(units=128, activation='relu'),
                            tf.keras.layers.Dropout(rate=0.3), # 드랍아웃 계층 생성 : rate로 이전 계층에서 제외할 뉴런의 비율 설정
                            tf.keras.layers.Dense(units=10, activation='softmax')
])

In [ ]:
# 컴파일 단계
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary() # 이전 실험보다 파라미터가 확연하게 감소 -> 풀링 계층의 효과

In [ ]:
# 신경망 학습 단계
history = model.fit(train_X, train_Y, epochs=500, validation_split=0.25, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')])

In [ ]:
# 정확도 및 손실값에 대한 결과 시각화
plt.figure(figsize = (12, 4))

plt.subplot(1, 2, 1) 
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='acc')
plt.plot(history.history['val_accuracy'], 'k--',label='val_acc')
plt.xlabel('Epoch')
plt.legend()

plt.show() # 과적합 현상 해소 확인 (약간 남아 있음이 보임)

In [ ]:
# 최종 평가
# 하이퍼 파라미터는 동일 / 정확도 및 손실값 어느정도 향상
# 풀링 계층을 활용하여 퍼셉트론 수 감축 -> 학습 속도 개선
# 과적합 현상 발생을 억제하기 위해 정규화 기법인 드랍아웃 적용 -> 일정 부분 과적합 현상 해소
# 최대치 풀링과 평균치 풀링 활용
model.evaluate(test_X, test_Y)

### 1.5 세 번째 실험 - VGGNet 스타일 적용
- Style Transfer 논문에서도 VGGNet활용
- VGGNet 참고 문헌 : https://arxiv.org/pdf/1709.01921.pdf
- Style Transfer 참고 문헌 : https://arxiv.org/pdf/1508.06576.pdf

In [ ]:
# VGGNet 스타일 일부 적용
# 합성곱 계층 2개 적용 / 풀링 레이어 삽입 / 합성곱 계층 2개 적용 / 풀링 레이어 삽입 / 완전연결계층 적용 / 완전연결계층 적용 / 완전연결계층 적용
model= tf.keras.Sequential([
                            tf.keras.layers.Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=32,
                                                    padding='same', activation='relu'),
                            tf.keras.layers.Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=64,
                                                    padding='same', activation='relu'),
                            tf.keras.layers.MaxPool2D(strides=(2, 2)),
                            tf.keras.layers.Dropout(rate=0.5),
                            tf.keras.layers.Conv2D(kernel_size=(3, 3),filters=128, padding='same', activation='relu'),
                            tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
                            tf.keras.layers.MaxPool2D(strides=(2, 2)),
                            tf.keras.layers.Dropout(rate=0.5),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(units=512, activation='relu'),
                            tf.keras.layers.Dropout(rate=0.5),
                            tf.keras.layers.Dense(units=256, activation='relu'),
                            tf.keras.layers.Dropout(rate=0.5),
                            tf.keras.layers.Dense(units=10, activation='softmax')
])

In [ ]:
# 컴파일 단계
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.summary() # 풀링 계층을 거쳤음에도 불구하고, '레이어가 깊게 쌓인 결과' 파라미터 다수 생성
                # -> 레이어가 깊게 쌓인만큼 높은 성능 기대 / 컴퓨팅 파워 중요

In [ ]:
# 신경망 학습 단계
history = model.fit(train_X, train_Y, epochs=500, validation_split=0.25, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss')])

In [ ]:
# 정확도 및 손실값에 대한 결과 시각화
plt.figure(figsize = (12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2) 
plt.plot(history.history['accuracy'], 'g-', label='acc')
plt.plot(history.history['val_accuracy'], 'k--',label='val_acc')
plt.xlabel('Epoch')
plt.legend()

plt.show() # 과적합 현상 해결

In [ ]:
# 최종 평가
# 하이퍼 파라미터는 동일 / 정확도 및 손실값 어느 정도 향상
# VGGNet 스타일 적용 -> 깊은 레이어만큼 다수의 파라미터 생성
# 과적합 현상 발생을 억제하기 위해 정규화 기법인 드랍아웃 적용 -> 과적합 현상 해소
# 90% 이상의 정확도 달성
model.evaluate(test_X, test_Y)

### 1.6 네 번째 실험 - VGGNet + 이미지 보강
- 이미지 보강 : 이미지의 크기, 각도 등을 변경하여 데이터로 추가

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 이미지 추가 생성 연습 (예시)
image_generator = ImageDataGenerator(
    rotation_range=10, # 이미지 회전값
    zoom_range=0.10, # 이미지 일부 확대
    shear_range=0.5, # 이미지 기울기
    width_shift_range=0.10, # 좌우 이동
    height_shift_range=0.10, # 상하 이동
    horizontal_flip=True, # 이미지 가로 뒤집기
    vertical_flip=False # 이미지 세로 뒤집기
)

augment_size = 100 # 한 번에 생성할 이미지 수

# flow()는 실제 보강된 이미지를 생성하는 함수로 Iterator 라는 객체를 생성한다.
# 이 객체에서는 값을 순서대로 꺼낼 수 있는데, 그 방법으로 next()를 사용하여 꺼낸다.
# 보강된 이미지들이 첫 번째에 위치해 있기 때문에 [0]을 할당하여 꺼내주고, 한 번에 꺼내는 이미지는 100장이 된다.
x_augment = image_generator.flow(x=np.tile(A=train_X[0].reshape(28*28), reps=100).reshape(-1, 28, 28, 1), # np.tile : A를 reps에 정해진 형식만큼 반복
                                  y=np.zeros(augment_size), # 라벨값은 딱히 줄 필요가 없기에 np.zeros() 할당
                                  batch_size=augment_size, # 배치 사이즈
                                  shuffle=False).next()[0] # next()로서 실제 값을 꺼냄
print(x_augment.shape)

x_augmented_1 = image_generator.flow(x=x_augment,
                                     y=np.zeros(augment_size),
                                     batch_size=augment_size,
                                     shuffle=False).next()[1]
print(x_augmented_1.shape)

In [ ]:
# 이미지 보강 확인
plt.figure(figsize=(10,10))

for i in range(100): # 각각 반복하여 이미지를 출력
    plt.subplot(10,10,i+1)
    plt.axis('off') # 축에 대한 정보 끄기
    plt.imshow(x_augment[i].reshape(28,28), cmap='pink') # 차원 수를 reshape()로 재조정하여 출력

plt.show() # 약간씩 다른 이미지가 생성

In [ ]:
# 이미지 추가 생성
# 훈련 데이터의 50% 추가 생성 (60000 + 30000)
image_generator = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.10,
    shear_range=0.5,
    width_shift_range=0.10,
    height_shift_range=0.10,
    horizontal_flip=True,
    vertical_flip=False)

augment_size = 30000

# 원본 이미지 무작위 선택 및 데이터 복사 : 무작위 선택 시 효율성 증가
x_choice = np.random.choice(train_X.shape[0], size=augment_size, replace=False)
x_augmented = train_X[x_choice].copy()
y_augmented = train_Y[x_choice].copy()

# 이미지를 변형할 원본 이미지를 찾기 위한 함수 예제 (randint : 중복 허용 O / choice : 중복 허용 X)
print(np.random.randint(train_X.shape[0], size = augment_size))
print(np.random.choice(train_X.shape[0], size = augment_size, replace=False))

# 보강 이미지 데이터 생성
x_augmented = image_generator.flow(x=x_augmented,
                                   y=np.zeros(augment_size),
                                   batch_size=augment_size,
                                   shuffle=False).next()[0]

# 보강 이미지 확인
plt.figure(figsize=(10, 10))

for i in range(100):
    plt.subplot(10, 10, i+1)
    plt.axis('off')
    plt.imshow(x_augmented[i].reshape(28, 28), cmap='pink')

plt.show()

#데이터 합쳐주기
train_X = np.concatenate((train_X, x_augmented))
train_Y = np.concatenate((train_Y, y_augmented))

#보강 데이터 결합 확인
print(train_X.shape)

In [ ]:
# 신경망 모델 생성 - 가장 성능이 좋았던 VGGNet 스타일 적용
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(input_shape=(28, 28, 1),kernel_size=(3, 3),
                                                     filters=32,padding='same',activation='relu'),
                             tf.keras.layers.Conv2D(input_shape=(28, 28, 1),kernel_size=(3, 3),
                                                     filters=64,padding='same',activation='relu'),
                             tf.keras.layers.MaxPool2D(strides=(2, 2)),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding='same', activation='relu'),
                             tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
                             tf.keras.layers.MaxPool2D(strides=(2, 2)),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(units=512, activation='relu'),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Dense(units=256, activation='relu'),
                             tf.keras.layers.Dropout(rate=0.5),
                             tf.keras.layers.Dense(units=10, activation='softmax')
])

In [ ]:
# 컴파일 단계
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

# 신경망 학습 단계
history = model.fit(train_X, train_Y, epochs=500, validation_split=0.25,
                    callbacks = [tf.keras.callbacks.EarlyStopping(patience=10 , monitor='val_loss')]) 

In [ ]:
# 정확도 및 손실값에 대한 결과 시각화
plt.figure(figsize = (12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2) 
plt.plot(history.history['accuracy'], 'g-', label='acc')
plt.plot(history.history['val_accuracy'], 'k--',label='val_acc')
plt.xlabel('Epoch')
plt.legend()

plt.show() # 과적합 발생 -> 양질의 데이터가 아닌 것을 임의로 추기해서로 추정

In [ ]:
# 최종 평가
# 하이퍼 파라미터는 동일 / 정확도 및 손실값 어느 정도 향상
# VGGNet 스타일 적용 / 이미지 보강 기법 활용
# 과적합 현상 ㅂ라생 / 92% 이상의 정확도 달성
model.evaluate(test_X, test_Y)

In [ ]:
#예측 수행
pred_X = model.predict(test_X[[1]])

#예측 이미지 확인
plt.imshow(test_X[1].reshape(28,28), cmap = 'pink')
plt.show()

#예측 결과 확인
print(pred_X+1)
print("예측 수행 범주 :",test_Y[1])